In [ ]:
# Question: 1. Build a Flask app that scrapes data from multiple websites and displays it on your site.
# You can try to scrap websites like youtube , amazon and show data on output pages and deploy it on cloud
# platform

!pip install flask requests beautifulsoup4

from flask import Flask, render_template
import requests
from bs4 import BeautifulSoup

app = Flask(__name__)

@app.route('/')
def index():
  return render_template('index.html')

@app.route('/youtube')
def youtube():
  url = "https://www.youtube.com/feed/trending"
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')

  trending_videos = []
  for video in soup.find_all('a', id='video-title'):
    title = video.text.strip()
    link = 'https://www.youtube.com' + video['href']
    trending_videos.append({'title': title, 'link': link})

  return render_template('youtube.html', videos=trending_videos)

@app.route('/amazon')
def amazon():
  search_query = "python books"  # Replace with your desired search
  url = f"https://www.amazon.com/s?k={search_query}"
  headers = {'User-Agent': 'Mozilla/5.0'}  # Add a user agent
  response = requests.get(url, headers=headers)
  soup = BeautifulSoup(response.text, 'html.parser')

  products = []
  for item in soup.find_all('div', {'data-component-type': 's-search-result'}):
    title_elem = item.find('span', class_='a-size-medium a-color-base a-text-normal')
    price_elem = item.find('span', class_='a-offscreen')
    if title_elem and price_elem:
      title = title_elem.text.strip()
      price = price_elem.text.strip()
      products.append({'title': title, 'price': price})

  return render_template('amazon.html', products=products)

if __name__ == '__main__':
  app.run(debug=True)


In [ ]:
# Question: 2. Create a Flask app that consumes data from external APIs and displays it to users . Try to find an public API which will give you a data and based on that call it and deploy it on cloud platform

# This code requires flask to be installed.
!pip install flask

from flask import Flask, render_template
import requests

app = Flask(__name__)

@app.route('/')
def index():
  # Replace with your actual API endpoint and parameters
  api_url = "https://api.example.com/data"
  response = requests.get(api_url)
  data = response.json()
  return render_template('index.html', data=data)

if __name__ == '__main__':
  app.run(debug=True)


In [ ]:
# Question: 3. create a flask app,web page and Implement OAuth2 authentication to allow users to log in using their Google or Facebook accounts.

!pip install flask flask-oauthlib

from flask import Flask, redirect, url_for, session, request, render_template
from flask_oauthlib.client import OAuth

app = Flask(__name__)
app.secret_key = 'development' # Replace with a strong secret key

oauth = OAuth(app)

# Google OAuth configuration
google = oauth.remote_app(
    'google',
    consumer_key='YOUR_GOOGLE_CLIENT_ID',
    consumer_secret='YOUR_GOOGLE_CLIENT_SECRET',
    request_token_params={
        'scope': 'https://www.googleapis.com/auth/userinfo.email'
    },
    base_url='https://www.googleapis.com/oauth2/v1/',
    request_token_url=None,
    access_token_method='POST',
    access_token_url='https://accounts.google.com/o/oauth2/token',
    authorize_url='https://accounts.google.com/o/oauth2/auth',
)

# Facebook OAuth configuration (similar structure to Google)
facebook = oauth.remote_app(
    'facebook',
    # ... configure Facebook OAuth settings ...
)

@app.route('/')
def index():
    return render_template('login.html')

@app.route('/login/google')
def login_google():
    return google.authorize(callback=url_for('authorized_google', _external=True))

@app.route('/login/facebook')
def login_facebook():
    return facebook.authorize(callback=url_for('authorized_facebook', _external=True))

@app.route('/authorized/google')
def authorized_google():
    resp = google.authorized_response()
    if resp is None:
        return 'Access denied: reason=%s error=%s' % (
            request.args['error_reason'],
            request.args['error_description']
        )
    session['google_token'] = (resp['access_token'], '')
    me = google.get('userinfo')
    return 'Logged in as: ' + me.data['email']

@app.route('/authorized/facebook')
def authorized_facebook():
    # ... handle Facebook authorization callback ...

@google.tokengetter
def get_google_oauth_token():
    return session.get('google_token')

@facebook.tokengetter
def get_facebook_oauth_token():
    # ... get Facebook OAuth token ...

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
# Question : 4. Develop a recommendation system using Flask that suggests content to users based on their preferences.

# This is a simplified example and would require a more robust approach
# for a real-world application.

#

# Mock user preferences (replace with actual user data)
user_preferences = {
  "category": "technology",
  "keywords": ["python", "programming", "data science"]
}

@app.route('/recommendations')
def recommendations():
  # Logic to filter content based on user preferences
  recommended_content = []
  # Example: Filter YouTube videos based on keywords
  for video in trending_videos:
    if any(keyword in video['title'].lower() for keyword in user_preferences['keywords']):
      recommended_content.append(video)

  # Example: Filter Amazon products based on category and keywords
  for product in products:
    if user_preferences['category'] in product['title'].lower() and \
       any(keyword in product['title'].lower() for keyword in user_preferences['keywords']):
      recommended_content.append(product)

  return render_template('recommendations.html', content=recommended_content)
